In [3]:
import sys
import pickle
sys.path.append('/Users/Malachite/Documents/UW/ARA/ARA-Plumes')

from utils import *

## Learn without any smoothing

In [ ]:
frames_path = "/Users/Malachite/Documents/UW/ARA/ARA-Plumes/plume_videos/July_20/video_high_1/fixed_avg_frames"
orig_center = (1587, 1035)
# mean_array, var1_array, var2_array = find_center_of_mass(frames_path=frames_path,
#                                                          center=orig_center,
#                                                          num_of_circs=22,
#                                                          radii=50,
#                                                          boundary_ring=False,
#                                                          interior_ring=False,
#                                                          save_figs=False
#                                                          )

In [ ]:
# Save arrays to pickle file
# arrays_dict = {
#     'mean': mean_array,
#     'var1': var1_array,
#     'var2': var2_array
# }

# with open("unsmooth_arrays.pkl", 'wb') as f:
#     pickle.dump(arrays_dict, f)

## Learn Path after KDE smoothing in space and time

In [ ]:
frames_path = "/Users/Malachite/Documents/UW/ARA/ARA-Plumes/plume_videos/July_20/video_high_1/time_space_blur"
save_path = "blurred_path_frames"
orig_center = (1587,1035)

# mean_array, var1_array, var2_array = find_center_of_mass(frames_path=frames_path,
#                                                          center=orig_center,
#                                                          num_of_circs=22,
#                                                          radii=50,
#                                                          boundary_ring=False,
#                                                          interior_ring=False,
#                                                          save_path=save_path)

In [ ]:
# Save Arrays to pickle file
# arrays_dict = {
#     'mean': mean_array,
#     'var1': var1_array,
#     'var2': var2_array
# }

# with open("smooth_arrays.pkl", 'wb') as f:
#     pickle.dump(arrays_dict,f)


## Apply Smoothing to mean points

In [4]:
sig_vals = [1]

for sig in sig_vals:
    frames_path = "/Users/Malachite/Documents/UW/ARA/ARA-Plumes/plume_videos/July_20/video_high_1/time_space_blur"
    save_path = f"mean_smooth_frames_sig{sig}"
    orig_center = (1587,1035)
    print("Sigma:", sig)
    print("Learning paths...")
    mean_array, var1_array, var2_array = find_center_of_mass(frames_path=frames_path,
                                                            center=orig_center,
                                                            num_of_circs=22,
                                                            radii=50,
                                                            boundary_ring=False,
                                                            interior_ring=False,
                                                            save_path=save_path,
                                                            mean_smoothing=True,
                                                            sigma=sig)
    # Save arrays to pickle file
    arrays_dict = {
        "mean": mean_array,
        'var1': var1_array,
        'var2': var2_array
    }

    with open(f"mean_smooth_sig{sig}_arrays.pkl", 'wb') as f:
        pickle.dump(arrays_dict,f)

    # Creating gif
    print("Creating gif...")
    frames_dir = f"/Users/Malachite/Documents/UW/ARA/ARA-Plumes/plume_videos/July_20/video_high_1/mean_smooth_frames_sig{sig}"
    duration = 1
    frames_range = [218-2,830-2] 

    create_gif(frames_dir=frames_dir,
            duration=duration,
            frames_range=frames_range, 
            gif_name=f"kde_mean_animation_sig{sig}")


Sigma: 1
Learning paths...


100%|██████████| 1913/1913 [2:22:36<00:00,  4.47s/it]     


Creating gif...


100%|██████████| 612/612 [00:18<00:00, 33.38it/s]


# Create Gif
frames = 218 - 830

In [ ]:
# frames_dir = "/Users/Malachite/Documents/UW/ARA/ARA-Plumes/plume_videos/July_20/video_high_1/learned_path_frames"
# duration = 1
# frames_range = [218,830] 

# create_gif(frames_dir=frames_dir,
#            duration=duration,
#            frames_range=frames_range)

In [ ]:
# frames_dir = "/Users/Malachite/Documents/UW/ARA/ARA-Plumes/plume_videos/July_20/video_high_1/blurred_path_frames"
# duration = 1
# frames_range = [218-2,830-2] 

# create_gif(frames_dir=frames_dir,
#            duration=duration,
#            frames_range=frames_range, 
#            gif_name="kde_animation")

# Apply SINDy

In [ ]:
import pysindy as ps
from sklearn.preprocessing import StandardScaler

## Load in arrays

In [ ]:
with open("smooth_arrays.pkl",'rb') as f:
    loaded_arrays = pickle.load(f)

mean_array = loaded_arrays["mean"]
var1_array = loaded_arrays["var1"]
var2_array = loaded_arrays["var2"]


In [ ]:
data_to_learn = mean_array[216:841,:]
data_to_learn.shape

## Instantiate ensemble SINDy

### Apply SINDy to smoothe data

In [ ]:
frame_count = count_files("/Users/Malachite/Documents/UW/ARA/ARA-Plumes/plume_videos/July_20/video_high_1/frames",'png')
sec = 22
fps = frame_count//sec
fps

start_time = 0
end_time = frame_count//fps
t = np.linspace(start_time,end_time,frame_count)
t = t[216:840]
t.shape



In [ ]:
# SINDy model
feature_names = ['a', 'b', 'c']
ensemble_optimizer = ps.STLSQ()
model = ps.SINDy(feature_names=feature_names, optimizer=ensemble_optimizer)
model.fit(data_to_learn,t=t, ensemble=True, quiet=True)
model.print()

In [ ]:
# SINDy model NORMALIZED
scalar = StandardScaler()
normalized_data = scalar.fit_transform(data_to_learn)

feature_names = ['a', 'b', 'c']
ensemble_optimizer = ps.STLSQ()
model = ps.SINDy(feature_names=feature_names, optimizer=ensemble_optimizer)
model.fit(normalized_data,t=t, ensemble=True, quiet=True)
model.print()

### Apply SINDy to unsmoothed data

In [ ]:
with open("unsmooth_arrays.pkl", 'rb') as f:
    loaded_arrays = pickle.load(f)

mean_array = loaded_arrays["mean"]
var1_array = loaded_arrays["var1"]
var2_array = loaded_arrays["var2"]

In [ ]:
data_to_learn = mean_array[216+2:841+1,:]
print(data_to_learn.shape)

frame_count = count_files("/Users/Malachite/Documents/UW/ARA/ARA-Plumes/plume_videos/July_20/video_high_1/frames",'png')
sec = 22
fps = frame_count//sec
fps

start_time = 0
end_time = frame_count//fps
t = np.linspace(start_time,end_time,frame_count)
t = t[216+2:840+2]
print(t.shape)



In [ ]:
# SINDy model
feature_names = ['a', 'b', 'c']
ensemble_optimizer = ps.STLSQ()
model = ps.SINDy(feature_names=feature_names, optimizer=ensemble_optimizer)
model.fit(data_to_learn,t=t, ensemble=True, quiet=True)
model.print()

In [ ]:
scalar = StandardScaler()
normalized_data = scalar.fit_transform(data_to_learn)

feature_names = ['a', 'b', 'c']
ensemble_optimizer = ps.STLSQ()
model = ps.SINDy(feature_names=feature_names, optimizer=ensemble_optimizer)
model.fit(normalized_data,t=t, ensemble=True, quiet=True)
model.print()

In [ ]:
for index, coeff in enumerate(model.coef_list):
    if index > 12:
        print(f"i={index}")
        print(coeff[0],"\n")

In [ ]:
normalized_data